# Interactive Accumulator Models with `Evidently`

This notebook contains interactive visualisations for a number of evidence models
created using the [evidently](https://github.com/EoinTravers/Evidently) package for python.

If you're running this notebook on [mybinder.org](https://mybinder.org/),
and find things to be a little slow,
you can run the code on your own machine instead by downloading the 
[GitHub repository](https://github.com/EoinTravers/Evidently).

If you're unfamiliar with [Jupyter notebooks](https://jupyter.org/),
you can:
- Run each code cell by clicking on it to select it,
  then clicking the 
  [<i class="fa-step-forward fa"></i> Run]
  button in the toolbar above.
- Run everything by clicking [Cell] → [Run All]

Either way, some of the cells below will generate interactive plots with sliders attached.
Go ahead and play with the sliders, and see what happens!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import evidently

from ipywidgets import interact, FloatSlider
def fs(v, low, high, step, desc=''):
    return FloatSlider(value=v, min=low, max=high, step=step, description=desc, continuous_update=False)

# Simple Wald Model


In [ ]:
print(evidently.Wald.__doc__) # Print the documentation for this model

In [ ]:
def wald_simulation_plot(t0=1., v=.5, z=0, a=1., c=1.):
    model = evidently.Wald(pars=[t0, v, z, a, c], max_time=5)
    evidently.viz.visualise_model(model, model_type='wald', measure='raw')
    title = 'Start time = %.1f, Drift = %.1f, Start = %.1f, Threshold = %.1f' % (t0, v, z, a)
    plt.suptitle(title, y=1.01)
    plt.show()

interact(wald_simulation_plot,
         t0  = fs(1., 0, 5., .1,   'Start time'),
         v   = fs(.5, 0, 2., .1,   'Drift'),
         z   = fs(0, 0, .5, .1,   'Starting point'),
         a   = fs(.5, 0., 2., .1,   'Threshold'),
         c   = fs(.1, 0., 2., .1,   'Noise SD'));

# Simple Drift Diffusion Model


In [ ]:
print(evidently.Diffusion.__doc__)

In [ ]:
def ddm_simulation_plot(t0=1., v=.5, z=0., a=.5, c=.1):
    model = evidently.Diffusion(pars=[t0, v, z, a, c])
    evidently.viz.visualise_model(model)
    title = 't0 = %.1f, Drift = %.1f, Bias = %.1f, Threshold = %.1f' % (t0, v, z, a)
    plt.suptitle(title, y=1.01)
    plt.show()

interact(ddm_simulation_plot,
         t0  = fs(1., 0, 2., .1,   't0'),
         v   = fs(.5, 0, 2., .1,   'Drift'),
         z   = fs(-.5, -1., 1., .1,  'Bias'),
         a     = fs(.5, 0., 2., .1,   'Threshold'),
         c   = fs(.2, 0., 1., .1,   'Noise SD'));

# Hierarchical Drift Diffusion Model

In [ ]:
print(evidently.HDiffusion.__doc__)

In [ ]:
def hddm_simulation_plot(m_t0=1., m_v=.5, m_z=0., m_a=.5,
                         s_t0=.1, s_v=.1, s_z=.1, s_a=.1, c=.1):
    model = evidently.HDiffusion(pars=[m_t0, m_v, m_z, m_a, s_t0, s_v, s_z, s_a, c])
    evidently.viz.visualise_model(model, measure='means', threshold=m_a)
    title = 't0 = %.1f (%.1f), Drift = %.1f (%.1f), Bias = %.1f (%.1f), Threshold = %.1f (%.1f), Noise = %.1f' % (
        m_t0, s_t0, m_v, s_v, m_z, s_z, m_a, s_a, c)
    plt.suptitle(title, y=1.05)
    plt.show()

interact(hddm_simulation_plot,
    m_t0 = fs(1., 0., 1., .1, 'T0'),
    m_v  = fs(.5, -1., 1., .1, 'Drift'),
    m_z  = fs(-.5, -1., 1., .1, 'Bias'),
    m_a  = fs(1., 0., 2., .1, 'Threshold'),
    s_t0 = fs(.1, 0., 1., .1, 'SD T0'),
    s_v  = fs(.5, 0., 1., .1, 'SD Drift'),
    s_z  = fs(.1, 0., 1., .1, 'SD Bias'),
    s_a  = fs(.1, 0., 1., .1, 'SD Threshold'),
    c    = fs(.1, 0., 1., .1, 'Noise'));

# Race

In [ ]:
print(evidently.Race.__doc__)

In [ ]:
def race_simulation_plot(t1, v1, z1, c1, t2, v2, z2, c2, a):
    model = evidently.Race(pars=[t1, v1, z1, c1, t2, v2, z2, c2, a])
    evidently.viz.visualise_model(model, model_type='race', measure='means')
    title = 'Option #1: t0 = %.1f, Drift = %.1f, Bias = %.1f, Noise = %.1f' % (t1, v1, z1, c1)
    title += '\nOption #2: t0 = %.1f, Drift = %.1f, Bias = %.1f, Noise = %.1f' % (t2, v2, z2, c2)
    title += '\nThreshold: %.1f' % a    
    plt.suptitle(title, y=1.2)
    plt.show()

interact(race_simulation_plot,
         t1   = fs(1., 0, 2., .1,   '#1: t0'),
         v1   = fs(.5, 0, 2., .1,   '#1: Drift'),
         z1   = fs(0., 0, 1., .1,  '#1: Bias'),
         c1   = fs(.1, 0., 1., .1,   '#1: Noise SD'),
         t2   = fs(2., 0, 2., .1,   '#2: t0'),
         v2   = fs(.75, 0, 2., .1,   '#2: Drift'),
         z2   = fs(.2, 0., 1., .1,  '#2: Bias'),
         c2   = fs(.2, 0., 1., .1,   '#2: Noise SD'),
         a     = fs(.5, 0., 2., .1,  'Threshold'));

## Schurger Model

In [ ]:
print(evidently.models.Schurger.__doc__)

In [ ]:

def schurger_simulation_plot(t0=0., v=.5, k=.6, z=0., a=1., c=1.):
    model = evidently.models.Schurger(pars=[t0, v, k, z, a, c], max_time=20.)
    evidently.viz.visualise_schurger(model, rp_duration=3)
    title = 'Start time = %.1f, Input = %.1f, Decay = %.1f, Start = %.1f, Threshold = %.1f, Noise = %.1f' % (
        t0, v, k, z, a, c)
    plt.suptitle(title, y=1.05)
    plt.show()

interact(schurger_simulation_plot,
         t0  = fs(0., 0, 5., .1,   'Start time'),
         v   = fs(.5, 0, 2., .1,   'Drift'),
         k   = fs(.6, 0, 2., .1,   'Decay'),
         z   = fs(0, 0, 1., .1,   'Starting point'),
         a   = fs(1., 0., 2., .1,   'Threshold'),
         c   = fs(.1, 0., 1., .1,   'Noise SD'));